# Torre de Hanoi - A*

Este notebook resuelve el problema de la Torre de Hanoi utilizando el algoritmo A*.

In [ ]:
## Setup
import statistics
import psutil
import os

## 1. Introducción

El problema de la Torre de Hanoi consiste en mover una pila de discos de una clavija a otra, siguiendo estas reglas:

1.	Solo se puede mover un disco a la vez.
2.	Un disco más grande no puede colocarse sobre uno más pequeño.
3.	Solo se puede mover el disco superior de una pila.

El Algoritmo A*  es un algoritmo de búsqueda heurística que encuentra el camino más corto entre un nodo inicial y un nodo objetivo, utilizando una función heurística para estimar el costo restante. 

In [ ]:
import time
import heapq

class State:
    def __init__(self, pegs, moves=0, heuristic=0):
        self.pegs = pegs
        self.moves = moves
        self.heuristic = heuristic

    def __eq__(self, other):
        return self.pegs == other.pegs

    def __hash__(self):
        return hash(tuple(tuple(peg) for peg in self.pegs))

    def __lt__(self, other):
        return (self.moves + self.heuristic) < (other.moves + other.heuristic)

def move_disk(state, source, destination):
    new_pegs = [list(peg) for peg in state.pegs]
    disk = new_pegs[source].pop()
    new_pegs[destination].append(disk)
    return State(new_pegs)

def is_valid_move(state, source, destination):
    if not state.pegs[source]:
        return False
    if state.pegs[destination] and state.pegs[source][-1] > state.pegs[destination][-1]:
        return False
    return True

def is_goal(state, num_disks):
    return len(state.pegs[2]) == num_disks

def heuristic(state, num_disks):
    count = 0
    for peg_index, peg in enumerate(state.pegs):
        if peg_index != 2: 
            count += len(peg)
    return (2 ** count) - 1 if count > 0 else 0

def get_successors(state, num_disks):
    successors = []
    for source in range(3):
        for destination in range(3):
            if source != destination and is_valid_move(state, source, destination):
                new_state = move_disk(state, source, destination)
                new_state.moves = state.moves + 1
                new_state.heuristic = heuristic(new_state, num_disks)
                successors.append((new_state, (source, destination)))
    return successors

def a_star_search(initial_state, num_disks):
    open_set = []
    heapq.heappush(open_set, (initial_state.moves + initial_state.heuristic, initial_state, []))
    visited = set()

    while open_set:
        _, current_state, path = heapq.heappop(open_set)

        if is_goal(current_state, num_disks):
            return path

        if current_state in visited:
            continue

        visited.add(current_state)

        for successor, move in get_successors(current_state, num_disks):
            if successor not in visited:
                heapq.heappush(open_set, (
                    successor.moves + successor.heuristic,
                    successor,
                    path + [move]
                ))

    return None

def solve_tower_of_hanoi_a_star(num_disks):
    initial_pegs = [list(range(num_disks, 0, -1)), [], []]
    initial_state = State(initial_pegs, moves=0, heuristic=heuristic(State(initial_pegs), num_disks))
    start_time = time.time()
    solution = a_star_search(initial_state, num_disks)
    end_time = time.time()
    execution_time = end_time - start_time
    return solution, execution_time

num_disks = 5 
solution, execution_time = solve_tower_of_hanoi_a_star(num_disks)

if solution:
    print(f"Solución encontrada en {len(solution)} movimientos:")
    print(f"Tiempo de ejecución: {execution_time} segundos")
    pegs = [list(range(num_disks, 0, -1)), [], []]
    for i, move in enumerate(solution, 1):
        source, destination = move
        disk = pegs[source].pop()
        pegs[destination].append(disk)
        print(f"Movimiento {i}: {pegs}\n")
else:
    print("No se encontró solución.")


Solución encontrada en 31 movimientos:
Tiempo de ejecución: 0.0020380020141601562 segundos
Movimiento 1: [[5, 4, 3, 2], [], [1]]

Movimiento 2: [[5, 4, 3], [2], [1]]

Movimiento 3: [[5, 4, 3], [2, 1], []]

Movimiento 4: [[5, 4], [2, 1], [3]]

Movimiento 5: [[5, 4, 1], [2], [3]]

Movimiento 6: [[5, 4, 1], [], [3, 2]]

Movimiento 7: [[5, 4], [], [3, 2, 1]]

Movimiento 8: [[5], [4], [3, 2, 1]]

Movimiento 9: [[5], [4, 1], [3, 2]]

Movimiento 10: [[5, 2], [4, 1], [3]]

Movimiento 11: [[5, 2, 1], [4], [3]]

Movimiento 12: [[5, 2, 1], [4, 3], []]

Movimiento 13: [[5, 2], [4, 3], [1]]

Movimiento 14: [[5], [4, 3, 2], [1]]

Movimiento 15: [[5], [4, 3, 2, 1], []]

Movimiento 16: [[], [4, 3, 2, 1], [5]]

Movimiento 17: [[1], [4, 3, 2], [5]]

Movimiento 18: [[1], [4, 3], [5, 2]]

Movimiento 19: [[], [4, 3], [5, 2, 1]]

Movimiento 20: [[3], [4], [5, 2, 1]]

Movimiento 21: [[3], [4, 1], [5, 2]]

Movimiento 22: [[3, 2], [4, 1], [5]]

Movimiento 23: [[3, 2, 1], [4], [5]]

Movimiento 24: [[3, 2, 1], [

## 2. PEAS del problema

- **Rendimiento (Performance)**: Minimizar el número de movimientos para resolver el problema.
- **Entorno (Environment)**: Las tres clavijas y los discos.
- **Actuadores (Actuators)**: Mover discos entre clavijas.
- **Sensores (Sensors)**: Observar el estado actual de las clavijas y los discos.

## 3. Propiedades del entorno de trabajo

- Completamente observable
- Determinista
- Secuencial
- Estático
- Discreto
- Agente único

## 4. Elementos del problema

- **Estado**: Configuración actual de los discos en las clavijas
- **Espacio de estados**: Todas las posibles configuraciones de discos en las clavijas
- **Árbol de búsqueda**: Árbol que representa todos los movimientos posibles y los estados resultantes
- **Nodo de búsqueda**: Un nodo en el árbol de búsqueda, que representa un estado y el movimiento que llevó a él
- **Objetivo**: Todos los discos en la clavija de destino en el orden correcto
- **Acción**: Mover un disco de una clavija a otra
- **Frontera**: Conjunto de nodos no expandidos en el árbol de búsqueda

## 5. Complejidad del algoritmo A*

- **Complejidad temporal**: En el peor de los casos, es O(b^d)
- **Complejidad espacial**: O(b^d)

## 6. Análisis de rendimiento empírico

Ejecutaremos el algoritmo 10 veces mediremos el tiempo de ejecución y el uso de memoria.

In [ ]:
def measure(num_discs, num_exec=10):
    times = []
    memory = []
    
    for _ in range(num_exec):
        init = time.time()
        process = psutil.Process(os.getpid())
        mem_init = process.memory_info().rss
        
        solve_tower_of_hanoi_a_star(num_discs)
        
        end = time.time()
        mem_end = process.memory_info().rss
        
        times.append(end - init)
        memory.append(mem_end- mem_init)
    
    return {
        'tiempo_promedio': statistics.mean(times),
        'tiempo_desviacion': statistics.stdev(times),
        'memoria_promedio': statistics.mean(memory),
        'memoria_desviacion': statistics.stdev(memory)
    }

results = measure(5)
print(f"Tiempo promedio: {results['tiempo_promedio']:.4f} s ± {results['tiempo_desviacion']:.4f} s")
print(f"Memoria promedio: {results['memoria_promedio'] / 1024:.2f} KB ± {results['memoria_desviacion'] / 1024:.2f} KB")
print()

Tiempo promedio: 0.0022 s ± 0.0004 s
Memoria promedio: 16.40 KB ± 34.38 KB



## 7. Comparación con la solución óptima

Compararemos el número de movimientos encontrados por A* con la solución óptima (2^k - 1, donde k es el número de discos).

In [ ]:
import statistics

def compare_with_optimum(num_disks, num_exec=10):
    moves = []
    for _ in range(num_exec):
        solution, execution_time = solve_tower_of_hanoi_a_star(num_disks)
        if solution is not None:
            moves.append(len(solution))
        else:
            print(f"No solution found for {num_disks} disks in this execution.")
            continue
    if not moves:
        print(f"No solutions found in {num_exec} executions for {num_disks} disks.")
        return None
    average_moves = statistics.mean(moves)
    optimal = 2 ** num_disks - 1
    percentage_difference = ((average_moves - optimal) / optimal) * 100

    return {
        'average_moves': average_moves,
        'optimal': optimal,
        'percentage_difference': percentage_difference
    }

disks = 5

results = compare_with_optimum(disks)
if results:
    print(f"Movimientos promedio: {results['average_moves']:.2f}")
    print(f"Solución óptima: {results['optimal']}")
    print(f"Diferencia porcentual: {results['percentage_difference']:.2f}%")

Movimientos promedio: 31.00
Solución óptima: 31
Diferencia porcentual: 0.00%


## 8. Conclusión

En este notebook, hemos implementado y analizado el algoritmo de búsqueda A* para resolver el problema de la Torre de Hanoi. Hemos examinado sus propiedades, rendimiento y lo hemos comparado con la solución óptima.

Nuestros resultados muestran que:

1.	El A* es altamente efectivo para resolver el problema de la Torre de Hanoi.
2.	A* puede manejar eficientemente instancias con 5 discos o más, encontrando soluciones en tiempos razonables.

Esta eficiencia se debe a varios factores:

1.	Uso de una heurística admisible y consistente: La heurística guía la búsqueda hacia los estados más prometedores, estimando correctamente el costo restante para alcanzar el objetivo sin sobreestimarlo.
2.	Priorización de estados óptimos: El A* prioriza los estados con menor costo total estimado (f(n) = g(n) + h(n)), enfocándose en caminos que conducen más rápidamente al objetivo.
3.	Reducción del espacio de búsqueda: Al evitar explorar caminos subóptimos y ciclos, el A* reduce significativamente el número de estados que necesita examinar, optimizando el uso de recursos.

En conclusión, mientras que el algoritmo A* es una opción práctica y eficiente para resolver el problema de la Torre de Hanoi con un número pequeño a moderado de discos, su eficiencia disminuye con problemas de mayor tamaño debido al crecimiento exponencial del espacio de búsqueda y las limitaciones de la heurística. 